<a href="https://colab.research.google.com/github/aleschiavo94/Sketches-Web-Search-Engine/blob/main/code/lsh_finetuning_bucket_dispersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Setting up the project

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import numpy as np
import time
import itertools
import os, shutil
import tensorflow as tf
import matplotlib.pyplot as plt

from random import random
from ipywidgets import Image
from numpy.linalg import norm
from IPython.display import display
from keras.models import Model
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint

# tensorflow version 2.4.0
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, InceptionV3

In [ ]:
zip_path = '/content/gdrive/Shareddrives/COMPUTER_VISION/MIM_zipped.zip'
!cp "{zip_path}" .
!unzip -q MIM_zipped.zip
!rm MIM_zipped.zip

In [ ]:
test_images_per_class = 20 # le immagini sono al massimo 80
train_images_per_class = 80 - test_images_per_class

In [ ]:
BASE_DIR = "/content/content/gdrive/Shareddrives/COMPUTER_VISION/MIRCV"
SKETCHES_DIR = os.path.join(BASE_DIR, "sketches")
MIRFLICKR_DIR = os.path.join(BASE_DIR, "mirflickr/mirflickr25k")

print(os.listdir(os.path.join(SKETCHES_DIR, "png")))
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)
      
copytree(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/all')

try:
  os.rename(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/train')
except:
  print('already renamed')

if not os.path.exists(f'{SKETCHES_DIR}/test'):
  os.mkdir(f'{SKETCHES_DIR}/test')
  for class_folder in os.listdir(f'{SKETCHES_DIR}/train'):
    if os.path.isdir(f'{SKETCHES_DIR}/train/{class_folder}'):
      images = os.listdir(f'{SKETCHES_DIR}/train/{class_folder}')
      os.mkdir(f'{SKETCHES_DIR}/test/{class_folder}')
      for i in range(test_images_per_class):
        os.rename(f'{SKETCHES_DIR}/train/{class_folder}/{images[i]}', f'{SKETCHES_DIR}/test/{class_folder}/{images[i]}')

['tractor', 'trombone', 'megaphone', 'microphone', 'wine-bottle', 'foot', 'shoe', 'pretzel', 'leaf', 'house', 'sailboat', 'hot air balloon', 't-shirt', 'piano', 'octopus', 'mermaid', 'motorbike', 'pipe (for smoking)', 'revolver', 'rainbow', 'pigeon', 'vase', 'stapler', 'mouse (animal)', 'scissors', 'rifle', 'umbrella', 'panda', 'potted plant', 'teddy-bear', 'grenade', 'brain', 'bush', 'snake', 'hourglass', 'train', 'giraffe', 'spoon', 'cup', 'comb', 'fire hydrant', 'sheep', 'mushroom', 'pumpkin', 'socks', 'moon', 'computer-mouse', 'mouth', 'standing bird', 'speed-boat', 'key', 'pineapple', 'candle', 'carrot', 'cat', 'flower with stem', 'santa claus', 'suitcase', 'hedgehog', 'mug', 'pizza', 'flying saucer', 'snowman', 'scorpion', 'cactus', 'apple', 'wrist-watch', 'seagull', 'power outlet', 'bowl', 'squirrel', 'blimp', 'feather', 'race car', 'tablelamp', 'camel', 'binoculars', 'laptop', 'cabinet', 'rabbit', 'nose', 'loudspeaker', 'keyboard', 'strawberry', 'ant', 'bicycle', 'hamburger', '

In [ ]:
BASE_DIR = "/content/content/gdrive/Shareddrives/COMPUTER_VISION/MIRCV"
SKETCHES_DIR = os.path.join(BASE_DIR, "sketches")
MIRFLICKR_DIR = os.path.join(BASE_DIR, "mirflickr/mirflickr25k")

# LSH definition

In [ ]:
class LSH:
  def __init__(self, feature_dim, g = 10, h = 20, w = 4, bitwise_hash = False):
    """
    We have to find a way to load the stored index if exists or initialize the 
    initial structure
    """
    self._index = {}
    if bitwise_hash:
      # self.x = np.random.randn(g, h, feature_dim) # non sembra richieda una distribuzione normale
      self.x = np.random.normal(size=(g, h, feature_dim))
    else:
      self.x = np.random.normal(size=(g, h, feature_dim))
    self.w = np.ones((g, h, 1)) * w
    self.b = np.random.rand(g, h, 1) * w
    self.bitwise_hash = bitwise_hash # another way to create h (h will be only 0 o 1)

  def _hash(self, features):
    """
    crea l'hash di più cose contemporaeamente si aspetta un array composto dalle features una sotta l'altra (linguaggio super matematico)
    """
    # g = np.trunc((np.dot(p, self.x) + self.b) / self.w)
    #g = np.trunc((np.dot(self.x, p) + self.b) / self.w) questo funziona con 1
    # g = np.transpose(np.trunc(((np.dot(self.x, p.T) + self.b) / self.w)), (0,2,1)) miglior modo di vederlo
    if self.bitwise_hash:
      return (np.transpose(np.dot(self.x, features.T), (0,2,1)) > 0).astype(int).astype(str)
    return np.transpose(np.trunc(((np.dot(self.x, features.T) + self.b) / self.w)), (0,2,1)).astype(int).astype(str)

  def insert(self, features, ids, labels):
    """
    Insert new data, ci aspettiamo un array d
    """
    g = self._hash(features)
    assert features.shape[0] == len(ids), "mismatch between ids length and features"
    assert len(labels) == len(ids), "mismatch between ids length and labels"
    
    number_elements = len(ids)
    i = 0
    # print("hash calculated")
    # print(g.shape)
    g_index = -1
    for g_function in g:
      start_inner_for = time.time()
      g_index += 1
      for row in g_function:
        if i % 10000 == 0:
          start = time.time()
        
        bucket_id = str(g_index) + '_' + ','.join(row)
        
        if i % 10000 == 0:
            end = time.time()
            # print(f'join {end - start}')
        if not bucket_id in self._index:
          # self._index[bucket_id] = { 'features': np.array([features[i % number_elements]]), 'ids': np.array([ids[i % number_elements]]), 'labels': np.array([labels[i % number_elements]])}
          self._index[bucket_id] = { 'features': [features[i % number_elements]], 'ids': [ids[i % number_elements]], 'labels': [labels[i % number_elements]]}
          if i % 10000 == 0:
            end = time.time()
            # print(f'not in bucket {end - start}')
        else:
          if ids[i % number_elements] in self._index[bucket_id]['ids']:
            # print("duplicate")
            continue
          if i % 10000 == 0:
            end = time.time()
            # print(f'checking duplicates {end - start}')
          # print("collision inserted")
          # self._index[bucket_id]['features'] = np.vstack((self._index[bucket_id]['features'], features[i % number_elements]))
          self._index[bucket_id]['features'].append(features[i % number_elements])
          # self._index[bucket_id]['ids'] = np.vstack((self._index[bucket_id]['ids'], ids[i % number_elements]))
          self._index[bucket_id]['ids'].append(ids[i % number_elements])
          # self._index[bucket_id]['labels'] = np.vstack((self._index[bucket_id]['labels'], labels[i % number_elements]))
          self._index[bucket_id]['labels'].append(labels[i % number_elements])
          if i % 10000 == 0:
            end = time.time()
            # print(f'stacking {end - start}')
        i += 1
        assert i > 0, 'out of bound'
      end_inner_for = time.time()
      # print(f'inner for time: {end_inner_for - start_inner_for}')
    
    for bucket_id in self._index:
      self._index[bucket_id]['features'] = np.array(self._index[bucket_id]['features'])
      # print(self._index[bucket_id]['features'].shape)
      self._index[bucket_id]['ids'] = np.array(self._index[bucket_id]['ids'], )
      self._index[bucket_id]['labels'] = np.array(self._index[bucket_id]['labels'])
      

  def query(self, features, top_k, mode = 'euclidean', return_cost = False):
    """
    Query the data
    """
    g = self._hash(np.array([features]))
    i = 0
    k = None
    top_k += 1 # per far ritornare k e non k - 1
    cost = 0
    g_index = -1
    assert mode in ['similarity', 'euclidean'], "mode must be similarity or euclidean"
    for g_function in g:
      g_index += 1
      for row in g_function:
        bucket_id = str(g_index) + '_' + ','.join(row)
        # print(bucket_id)
        if bucket_id in self._index:
          # posso avere duplicati perchè se i punti vengono inseriti in più bucket, posso avere duplicati
          # quindi devo eliminarli
          # l'ho messo qua fuori che il controllo duplicati è uguale per tutte e due le distanze
          # print(f'bucket {bucket_id}')
          if k is not None:
            # print("duplicate")
            duplicate_index = np.isin(self._index[bucket_id]['ids'], k['ids'])
            if duplicate_index.all():
              continue; # se sono tutti duplicati non ha senso contare nulla
            bucket = {}
            bucket['ids'] = self._index[bucket_id]['ids'][~duplicate_index] # prendo quelli che non sono duplicati
            # print(duplicate_index)
            # print(self._index[bucket_id]['features'])
            bucket['features'] = self._index[bucket_id]['features'][~duplicate_index.flatten()] # each duplicate index must delete a row of features
            bucket['labels'] = self._index[bucket_id]['labels'][~duplicate_index]
          else:
            bucket = self._index[bucket_id]
        
          if mode == 'euclidean':
            # print(bucket['features'].shape)
            dist = norm(bucket['features'] - np.array(features), axis=1)
            # print(f'dist shape {dist.shape} and dist size {dist.size}')
            cost += dist.size
            if k is None:
              idx_partitioned = np.argpartition(dist, top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0] - 1)
              if dist.shape[0] - 1 > top_k - 1:  
                  idx_partitioned = idx_partitioned[:top_k - 1]
              k = {}
              # qua è più comodo avere array 1- dimensionali
              k['ids'] = bucket['ids'][idx_partitioned].flatten()
              k['labels'] = bucket['labels'][idx_partitioned].flatten()
              k['distances'] = dist[idx_partitioned]
              continue
            # https://stackoverflow.com/questions/10337533/a-fast-way-to-find-the-largest-n-elements-in-an-numpy-array
            # argpartition sembra essere incredibilmente veloce
            # ma non ordina completamente, ordina solo rispetto un punto, nel senso
            # io gli sto dicendo butta quelli più piccoli di k da una parte e quelli più grandi all'altra, ma non sto ordinando
            if k['distances'].shape[0] < top_k:
                # print((k['distances'].shape, dist.shape))
                distances = np.concatenate((k['distances'], dist))
                # print((k['ids'].shape, bucket['ids'].shape))
                # print(k['ids'])
                # print(bucket['ids'])
                ids = np.concatenate((k['ids'], bucket['ids']))
                # print((k['labels'].shape, bucket['labels'].shape))
                labels = np.concatenate((k['labels'], bucket['labels']))
                idx_sorted = np.argpartition(distances, top_k - 1 if distances.shape[0] - 1 > top_k else distances.shape[0] - 1)
                if distances.shape[0] - 1 > top_k - 1:  
                  idx_sorted = idx_sorted[:top_k - 1]
                k['ids'] = ids[idx_sorted]
                k['labels'] = labels[idx_sorted]
                k['distances'] = distances[idx_sorted]
                # print(f'k = {k}')
                continue

            idx = dist < np.max(k['distances'])
            # print(f"idx = {idx}")
            if np.any(idx):
              distances = np.concatenate((k['distances'], dist[idx]))
              ids = np.concatenate((k['ids'], bucket['ids'][idx]))
              labels = np.concatenate((k['labels'], bucket['labels'][idx]))
              idx_sorted = np.argpartition(distances, top_k - 1 if distances.shape[0] - 1 > top_k else distances.shape[0] - 1)
              if distances.shape[0] - 1 > top_k - 1:  
                  idx_sorted = idx_sorted[:top_k - 1]
              k['ids'] = ids[idx_sorted]
              k['labels'] = labels[idx_sorted]
              k['distances'] = distances[idx_sorted]

          else:
            # print(bucket['features'].shape)
            sim = np.sum(bucket['features'] * np.array(features), axis=1) / (norm(bucket['features'], axis=1) * norm(np.array([features]), axis=1))
            # print(f'sim shape {sim.shape} and sim size {sim.size}')
            cost += sim.size
            if k is None:
              idx_partitioned = np.argpartition(sim, -(top_k - 1) if sim.shape[0] - 1 > top_k - 1 else sim.shape[0] - 1)
              if sim.shape[0] - 1 > top_k - 1:  
                idx_partitioned = idx_partitioned[-(top_k - 1):]
              k = {}
              # qua è più comodo avere array 1- dimensionali
              k['ids'] = bucket['ids'][idx_partitioned].flatten()
              k['labels'] = bucket['labels'][idx_partitioned].flatten()
              k['similarities'] = sim[idx_partitioned]
              continue
            # https://stackoverflow.com/questions/10337533/a-fast-way-to-find-the-largest-n-elements-in-an-numpy-array
            # argpartition sembra essere incredibilmente veloce
            # ma non ordina completamente, ordina solo rispetto un punto, nel senso
            # io gli sto dicendo butta quelli più piccoli di k da una parte e quelli più grandi all'altra, ma non sto ordinando
            if k['similarities'].shape[0] < top_k:
                # print((k['similarities'].shape, sim.shape))
                similarities = np.concatenate((k['similarities'], sim))
                # print((k['ids'].shape, bucket['ids'].shape))
                # print(k['ids'])
                # print(bucket['ids'])
                ids = np.concatenate((k['ids'], bucket['ids']))
                # print((k['labels'].shape, bucket['labels'].shape))
                labels = np.concatenate((k['labels'], bucket['labels']))
                idx_sorted = np.argpartition(similarities, -(top_k - 1) if similarities.shape[0] - 1 > top_k - 1 else similarities.shape[0] - 1)
                if similarities.shape[0] - 1 > top_k - 1:  
                  idx_sorted = idx_sorted[-(top_k - 1):]
                k['ids'] = ids[idx_sorted]
                k['labels'] = labels[idx_sorted]
                k['similarities'] = similarities[idx_sorted]
                # print(f'k = {k}')
                continue

            idx = sim > np.min(k['similarities'])
            # print(f"idx = {idx}")
            if np.any(idx):
              similarities = np.concatenate((k['similarities'], sim[idx]))
              ids = np.concatenate((k['ids'], bucket['ids'][idx]))
              labels = np.concatenate((k['labels'], bucket['labels'][idx]))
              idx_sorted = np.argpartition(similarities, -(top_k - 1) if similarities.shape[0] - 1 > top_k - 1 else similarities.shape[0] - 1)
              if similarities.shape[0] - 1 > top_k - 1:  
                idx_sorted = idx_sorted[-(top_k - 1):]
              k['ids'] = ids[idx_sorted]
              k['labels'] = labels[idx_sorted]
              k['similarities'] = similarities[idx_sorted]
        i += 1
    # ora ordino totalmente i risultati
    if k is None:
      if return_cost:
        return (None, 1)
      return None #zero result
    if mode == 'euclidean':
      idx_sorted = np.argsort(k['distances'])
      idx_sorted = idx_sorted[:top_k - 1 if k['distances'].shape[0] - 1 > top_k else k['distances'].shape[0]]
      k['distances'] = k['distances'][idx_sorted]
      k['ids'] = k['ids'][idx_sorted]
      k['labels'] = k['labels'][idx_sorted]
      if return_cost:
        return (k, cost)
      return k
    idx_sorted = np.argsort(k['similarities'])[::-1]
    idx_sorted = idx_sorted[:top_k - 1 if k['similarities'].shape[0] - 1 > top_k else k['similarities'].shape[0]]
    k['similarities'] = k['similarities'][idx_sorted]
    k['ids'] = k['ids'][idx_sorted]
    k['labels'] = k['labels'][idx_sorted]
    if return_cost:
      return (k, cost)
    return k

  def store(self):
    pass


## No Index

In [ ]:
# Useful to have comparable performance
class NO_INDEX:
  def __init__(self):
    self.features = None
    self.ids = None
    self.labels = None

  def insert(self, features, ids, labels):
    assert features.shape[0] == len(ids), "mismatch between ids length and features"
    assert len(labels) == len(ids), "mismatch between ids length and labels"
    self.features = features
    self.ids = ids
    self.labels = labels
  
  def query(self, features, top_k, mode = 'euclidean', return_cost = False):
    assert mode in ['similarity', 'euclidean'], "mode must be similarity or euclidean"
    top_k += 1
    if mode == 'euclidean':
      dist = norm(self.features - np.array(features), axis=1)
      idx_partitioned = np.argpartition(dist, top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0] - 1)
      k = {}
      # qua è più comodo avere array 1- dimensionali
      k['ids'] = self.ids[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['labels'] = self.labels[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['distances'] = dist[idx_partitioned][:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      
      idx_sorted = np.argsort(k['distances'])
      idx_sorted = idx_sorted
      k['distances'] = k['distances'][idx_sorted]
      k['ids'] = k['ids'][idx_sorted]
      k['labels'] = k['labels'][idx_sorted]
      if return_cost:
        return (k, dist.size)
      return k
    sim = np.sum(self.features * np.array(features), axis=1) / (norm(self.features, axis=1) * norm(np.array([features]), axis=1))
    idx_partitioned = np.argpartition(sim, -(top_k - 1) if sim.shape[0] - 1 > top_k - 1 else sim.shape[0] - 1)
    k = {}
    # qua è più comodo avere array 1- dimensionali
    if sim.shape[0] - 1 > top_k - 1:  
      idx_partitioned = idx_partitioned[-(top_k - 1):]

    k['ids'] = self.ids[idx_partitioned].flatten()
    k['labels'] = self.labels[idx_partitioned].flatten()
    k['similarities'] = sim[idx_partitioned]
    idx_sorted = np.argsort(k['similarities'])[::-1]
    idx_sorted = idx_sorted
    k['similarities'] = k['similarities'][idx_sorted]
    k['ids'] = k['ids'][idx_sorted]
    k['labels'] = k['labels'][idx_sorted]
    if return_cost:
      return (k, sim.size)
    return k
    

## Evaluation utils

In [ ]:
def average_precision(requested_label, result_labels, n_ground_truth = 80):
  """
  label ricercata, label ottenute, il numero di oggetti che ci sono quella label
  """
  correct_array = (requested_label == result_labels).astype(int)
  precision_array = [np.mean(correct_array[:k]) for k in range(1, correct_array.shape[0] + 1)]
  # print(precision_array * correct_array) # mi rimangono solo quelli a 1
  return np.sum(precision_array * correct_array) / n_ground_truth


def mAP(index, features, n_queries = 250, n_labels = 250, img_per_labels = 80, mode = 'euclidean', check_first_label=True, n_ground_truth=None):
  sum = 0
  if not n_ground_truth:
    n_ground_truth = img_per_labels
  for i in range(n_queries):
    label = i % n_labels
    image_idx = ((i * img_per_labels) + int(random() * img_per_labels)) % (n_labels * img_per_labels)
    # print('QUERY')
    # print('index = ' + str(image_idx))
    # print('label =' + str(label))
    res = index.query(features[image_idx], n_ground_truth, mode = mode)
    if res is None:
      sum += 0
      continue
    # print('first label of resultset (must be equal to label) = ' + str(res['labels'][0]))
    if check_first_label:
      assert res['labels'][0] == label, 'deve essere della stessa label'
    a = average_precision(label, res['labels'], n_ground_truth)
    sum += a
  return sum / n_queries

def ie(cost_with_index, cost_no_index=45000): # utilizzando tutte le features abbiamo 45000 immagini tra distrattore e non
  return cost_no_index / cost_with_index


def mAP_with_ie(index, features, n_queries = 2500, n_labels = 250, img_per_labels = 80, mode = 'euclidean', check_first_label=True, n_ground_truth=None):
  sum = 0
  sum_ie = 0
  if not n_ground_truth:
    n_ground_truth = img_per_labels
  for i in range(n_queries):
    label = i % n_labels
    image_idx = ((i * img_per_labels) + int(random() * img_per_labels)) % (n_labels * img_per_labels)
    # print('QUERY')
    # print('index = ' + str(image_idx))
    # print('label =' + str(label))
    res, cost = index.query(features[image_idx], n_ground_truth, mode = mode, return_cost = True)
    if res is None:
      sum += 0
      sum_ie += ie(cost, n_ground_truth * n_labels)
      continue
    # print('first label of resultset (must be equal to label) = ' + str(res['labels'][0]))
    if check_first_label:
      assert res['labels'][0] == label, 'deve essere della stessa label'
    a = average_precision(label, res['labels'], n_ground_truth)
    sum += a
    sum_ie += ie(cost, n_ground_truth * n_labels)
  return { 'mAP': sum / n_queries, 'ie': sum_ie / n_queries }

# try to calculate bucket dispersion
# ritorna la percentuale media, la deviazione standard della classe più popolosa dei bucket,
# ritorna anche il numero di bucket
def bucket_dispersion(index):
  percs = None
  itemsCount = None
  i = 0
  for bucket in index._index:
    i += 1
    _, counts = np.unique(index._index[bucket]['labels'], return_counts=True)
    perc = np.max(counts) / np.sum(counts)
    itemCount = np.sum(counts)
    if percs is None:
      percs = np.array([perc])
      itemsCount = np.array([itemCount])
    else:
      percs = np.concatenate((percs, np.array([perc])))
      itemsCount = np.concatenate((itemsCount, np.array([itemCount])))
  return {
      'perc': {
          'mean': np.mean(percs),
          'deviation': np.std(percs)
      },
      'count': {
          'average': np.mean(itemsCount),
          'deviation': np.std(itemsCount)
      },
      'n_buckets': i,
      'n_items_counting_duplicates': np.sum(itemsCount) # ci sono anche i duplicati
  }


# Model loading and features extraction

In [ ]:
def extract_features(extractor, generator, sample_count, dim=2048):
  features = np.zeros((sample_count, dim)) #extractor output shape 
  i = 0
  for inputs_batch, labels_batch in generator:
    start = time.time()
    features_batch = extractor.predict(inputs_batch)
    start = time.time()
    if (i + 1) * BATCH_SIZE > sample_count:
      features[i * BATCH_SIZE : sample_count , :] = features_batch
      assert np.array_equal(np.argmax(labels_batch, axis = 1), generator.labels[i * BATCH_SIZE : sample_count]), 'LABELS NOT CORRESPONDING REINIZIALIZE GENERATOR'
    else:
      features[i * BATCH_SIZE : (i + 1) * BATCH_SIZE, : ] = features_batch
      assert np.array_equal(np.argmax(labels_batch, axis = 1), generator.labels[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]), 'LABELS NOT CORRESPONDING REINIZIALIZE GENERATOR'
    i += 1
    if i * BATCH_SIZE >= sample_count:
      break
  
  return features

In [ ]:
IMG_HEIGHT = 299
IMG_WIDTH = 299
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
BATCH_SIZE = 64


sketches_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
mirflickr_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

sketches_generator_train = sketches_datagen.flow_from_directory(
      # This is the target directory
      SKETCHES_DIR + '/train',
      shuffle=False,
      target_size=(IMG_HEIGHT, IMG_WIDTH),
      batch_size=BATCH_SIZE)

mirflickr_generator = mirflickr_datagen.flow_from_directory(
        # This is the target directory
        MIRFLICKR_DIR,
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)

sketches_generator_test = sketches_datagen.flow_from_directory(
      # This is the target directory
      SKETCHES_DIR + '/test',
      shuffle=False,
      target_size=(IMG_HEIGHT, IMG_WIDTH),
      batch_size=BATCH_SIZE)


Found 15000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.
Found 5000 images belonging to 250 classes.


In [ ]:
MODEL_PATH = "/content/gdrive/Shareddrives/COMPUTER_VISION/models"
model = models.load_model(MODEL_PATH + '/inception_finetuning_classification_3_last_one_more_train_all_parameters_more_train.h5')
extractor = Model(model.input, model.layers[-2].output)
extractor.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_376 (Conv2D)             (None, None, None, 3 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_376 (BatchN (None, None, None, 3 96          conv2d_376[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (None, None, None, 3 0           batch_normalization_376[0][0]    
______________________________________________________________________________________________

In [ ]:
sketches_features_train = extract_features(extractor, sketches_generator_train, 15000, dim=2048)
sketches_features_test = extract_features(extractor, sketches_generator_test, 5000, dim=2048)
mirflickr_features = extract_features(extractor, mirflickr_generator, 25000, dim=2048)

# NO Index mAP

In [ ]:
print('mAP on train')
no_index_base = NO_INDEX()
no_index_base.insert(np.vstack((sketches_features_train, mirflickr_features)), 
              np.concatenate((sketches_generator_train.filenames, mirflickr_generator.filenames)), 
              np.concatenate((sketches_generator_train.labels, np.array([250] * mirflickr_features.shape[0]))))
print('Euclidean mAP')
print(mAP(no_index_base, sketches_features_train, img_per_labels=train_images_per_class))
print('Cosine mAP')
print(mAP(no_index_base, sketches_features_train, mode = 'similarity', img_per_labels=train_images_per_class))
print('')

print('mAP using test as query on train')

print('Euclidean mAP')
print(mAP(no_index_base, sketches_features_test, img_per_labels=test_images_per_class, check_first_label=False, n_ground_truth=train_images_per_class))
print('Cosine mAP')
print(mAP(no_index_base, sketches_features_test, mode = 'similarity', img_per_labels=test_images_per_class, check_first_label=False, n_ground_truth=train_images_per_class))
print('')


del no_index_base

mAP on train
Euclidean mAP
0.39624772359313365
Cosine mAP
0.45834064304191197

mAP using test as query on train
Euclidean mAP
0.15376194037501312
Cosine mAP
0.18571864659363127

query cost always 45000


# LSH Finetuning

In [ ]:
def evaluate_lsh(index, train, train_generator, test, test_generator, distractor, distractor_generator):
  index.insert(np.vstack((train, distractor)), 
                np.concatenate((train_generator.filenames, distractor_generator.filenames)), 
                np.concatenate((train_generator.labels, np.array([250] * distractor.shape[0]))))
  print('Euclidean mAP')
  print(mAP_with_ie(index, train, img_per_labels=train_images_per_class))
  print('Cosine mAP')
  print(mAP_with_ie(index, train, mode = 'similarity', img_per_labels=train_images_per_class))
  print('')

  print('mAP using test as query on train')

  print('Euclidean mAP')
  print(mAP_with_ie(index, test, img_per_labels=test_images_per_class, check_first_label=False, n_ground_truth=train_images_per_class))
  print('Cosine mAP')
  print(mAP_with_ie(index, test, mode = 'similarity', img_per_labels=test_images_per_class, check_first_label=False, n_ground_truth=train_images_per_class))

  print('bucket dispersion')
  print(bucket_dispersion(index))

## LSH BitWise = False

### Test 1 - g = 3, h = 5

In [ ]:
evaluate_lsh(
  LSH(2048, g = 3, h = 5, w = 4, bitwise_hash = False),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
)

Euclidean mAP
{'mAP': 0.03614105046761578, 'ie': 2509.1967928607896}
Cosine mAP
{'mAP': 0.036512255789907896, 'ie': 2668.701430340313}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.01781238117182862, 'ie': 3363.512256555461}
Cosine mAP
{'mAP': 0.019087775077214635, 'ie': 3212.2137378473362}
bucket dispersion
{'perc': {'mean': 0.8614233958802622, 'deviation': 0.2559319332905669}, 'count': {'average': 5.408328826392645, 'deviation': 37.597062978644956}, 'n_buckets': 22188, 'n_items_counting_duplicates': 120000}


### Test 2 - g = 3, h = 3

In [ ]:
evaluate_lsh(
  LSH(2048, g = 3, h = 3, w = 4, bitwise_hash = False),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.08165711809693882, 'ie': 355.11893729147965}
Cosine mAP
{'mAP': 0.09192493891390532, 'ie': 310.7742971803881}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.059117561203445294, 'ie': 351.71106186083415}
Cosine mAP
{'mAP': 0.06982999386051736, 'ie': 297.5680065856871}
bucket dispersion
{'perc': {'mean': 0.7280029198080883, 'deviation': 0.3231802353363183}, 'count': {'average': 24.564994882292734, 'deviation': 183.97494634633577}, 'n_buckets': 4885, 'n_items_counting_duplicates': 120000}


### Test 3 - g = 5, h = 2

In [ ]:
evaluate_lsh(
  LSH(2048, g = 5, h = 2, w = 4, bitwise_hash = False),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.2204647065954188, 'ie': 5.791378578628736}
Cosine mAP
{'mAP': 0.25402304927068775, 'ie': 15.954885533930407}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.18913629637336996, 'ie': 11.528419837402286}
Cosine mAP
{'mAP': 0.21850763392782854, 'ie': 11.962597699520488}
bucket dispersion
{'perc': {'mean': 0.6229000107491875, 'deviation': 0.3375273410066464}, 'count': {'average': 123.76237623762377, 'deviation': 636.1747158053337}, 'n_buckets': 1616, 'n_items_counting_duplicates': 200000}


### Test 4 - g = 8, h = 2

In [ ]:
evaluate_lsh(
  LSH(2048, g = 8, h = 2, w = 4, bitwise_hash = False),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.2958958416119443, 'ie': 2.1656765411997707}
Cosine mAP
{'mAP': 0.3217574035967056, 'ie': 2.617434213347446}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.24537978324370485, 'ie': 1.9330499135929686}
Cosine mAP
{'mAP': 0.27101246746784263, 'ie': 2.6859283844460617}
bucket dispersion
{'perc': {'mean': 0.6188199434399093, 'deviation': 0.34939694662490905}, 'count': {'average': 125.58869701726844, 'deviation': 688.4948657204496}, 'n_buckets': 2548, 'n_items_counting_duplicates': 320000}


### Test 5 - g = 4, h = 2

In [ ]:
evaluate_lsh(
  LSH(2048, g = 4, h = 2, w = 4, bitwise_hash = False),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.21588422718849098, 'ie': 7.711432197670193}
Cosine mAP
{'mAP': 0.22439964912118568, 'ie': 34.99700216675116}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.17305548829052694, 'ie': 11.24306918007389}
Cosine mAP
{'mAP': 0.2038813317735243, 'ie': 14.281028230339931}
bucket dispersion
{'perc': {'mean': 0.620138418570836, 'deviation': 0.3443745541932192}, 'count': {'average': 134.68013468013467, 'deviation': 677.1461954975082}, 'n_buckets': 1188, 'n_items_counting_duplicates': 160000}


### Test 6 - g = 5, h = 1

In [ ]:
evaluate_lsh(
  LSH(2048, g = 5, h = 1, w = 4, bitwise_hash = False),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.3861241576610084, 'ie': 0.48644587959379126}
Cosine mAP
{'mAP': 0.42321074159335353, 'ie': 0.4842529660656094}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.3176951339401956, 'ie': 0.5213734379160141}
Cosine mAP
{'mAP': 0.3740915370939836, 'ie': 0.46881455402609296}
bucket dispersion
{'perc': {'mean': 0.5233422134508805, 'deviation': 0.3476517120159299}, 'count': {'average': 1333.3333333333333, 'deviation': 3571.9683381699915}, 'n_buckets': 150, 'n_items_counting_duplicates': 200000}


### Test 7 - g = 5, h = 2

In [ ]:
evaluate_lsh(
  LSH(2048, g = 5, h = 2, w = 4, bitwise_hash = False),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.21759190083725274, 'ie': 9.882893287174422}
Cosine mAP
{'mAP': 0.26525889655659435, 'ie': 10.341478581132685}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.19105903699880342, 'ie': 10.034816977814566}
Cosine mAP
{'mAP': 0.2226306465479609, 'ie': 11.814055386273653}
bucket dispersion
{'perc': {'mean': 0.617963758732887, 'deviation': 0.3452220621992816}, 'count': {'average': 122.77470841006753, 'deviation': 642.3146193080048}, 'n_buckets': 1629, 'n_items_counting_duplicates': 200000}


### Test 8 - g = 7, h = 2

In [ ]:
evaluate_lsh(
  LSH(2048, g = 7, h = 2, w = 4, bitwise_hash = False),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.2682402021683603, 'ie': 4.744836061747683}
Cosine mAP
{'mAP': 0.3126806303065159, 'ie': 3.4177092496684867}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.24010568714673594, 'ie': 5.82042023878564}
Cosine mAP
{'mAP': 0.2633255584664252, 'ie': 3.936713184221218}
bucket dispersion
{'perc': {'mean': 0.6204915256274679, 'deviation': 0.3438883216464238}, 'count': {'average': 120.74169900819318, 'deviation': 722.8325207637048}, 'n_buckets': 2319, 'n_items_counting_duplicates': 280000}


## LSH BitWise = True

### Test 1 - g = 3, h = 5

In [ ]:
evaluate_lsh(
  LSH(2048, g = 3, h = 5, w = 4, bitwise_hash = True),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.316680306859603, 'ie': 4.6274673446053365}
Cosine mAP
{'mAP': 0.395348767529745, 'ie': 4.705326849755457}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.30303350598635076, 'ie': 5.0142654416680275}
Cosine mAP
{'mAP': 0.33097258699155985, 'ie': 4.667283156296233}
bucket dispersion
{'perc': {'mean': 0.25631335073856415, 'deviation': 0.2691370596539978}, 'count': {'average': 1250.0, 'deviation': 2733.230319451083}, 'n_buckets': 96, 'n_items_counting_duplicates': 120000}


### Test 2 - g = 3, h = 6

In [ ]:
evaluate_lsh(
  LSH(2048, g = 3, h = 6, w = 4, bitwise_hash = True),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.33180048956907937, 'ie': 7.504740666873613}
Cosine mAP
{'mAP': 0.34969790509415893, 'ie': 7.442158447596275}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.283275594317394, 'ie': 7.952548826744177}
Cosine mAP
{'mAP': 0.3184283303706314, 'ie': 8.174635084376439}
bucket dispersion
{'perc': {'mean': 0.29811823859565706, 'deviation': 0.24227334299018607}, 'count': {'average': 625.0, 'deviation': 1502.5304558754653}, 'n_buckets': 192, 'n_items_counting_duplicates': 120000}


### Test 3 - g = 5, h = 2


In [ ]:
evaluate_lsh(
  LSH(2048, g = 5, h = 2, w = 4, bitwise_hash = True),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.4360485546174223, 'ie': 0.31182855868160375}
Cosine mAP
{'mAP': 0.5018093840798531, 'ie': 0.31420188886133155}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.36432599189614456, 'ie': 0.30799421923154563}
Cosine mAP
{'mAP': 0.42424155063966995, 'ie': 0.311769659469289}
bucket dispersion
{'perc': {'mean': 0.3496864073807303, 'deviation': 0.3640991588502488}, 'count': {'average': 10000.0, 'deviation': 8618.353003909739}, 'n_buckets': 20, 'n_items_counting_duplicates': 200000}


### Test 4 - g = 5, h = 5

In [ ]:
evaluate_lsh(
  LSH(2048, g = 5, h = 5, w = 4, bitwise_hash = True),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.37553513603664646, 'ie': 2.336255941256936}
Cosine mAP
{'mAP': 0.4472989764249673, 'ie': 2.0826454998583728}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.3133187491027705, 'ie': 2.23716949603222}
Cosine mAP
{'mAP': 0.39067410533763686, 'ie': 2.1967354790895435}
bucket dispersion
{'perc': {'mean': 0.2912285086152695, 'deviation': 0.2640175545365738}, 'count': {'average': 1250.0, 'deviation': 2347.002985191966}, 'n_buckets': 160, 'n_items_counting_duplicates': 200000}


### Test 5 - g = 4, h = 4

In [ ]:
evaluate_lsh(
  LSH(2048, g = 4, h = 4, w = 4, bitwise_hash = True),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

NameError: ignored

### Test 6 - g = 3, h = 1

In [ ]:
evaluate_lsh(
  LSH(2048, g = 3, h = 1, w = 4, bitwise_hash = True),
  sketches_features_train, 
  sketches_generator_train, 
  sketches_features_test, 
  sketches_generator_test, 
  mirflickr_features, 
  mirflickr_generator
) # meglio passarlo così che elimina subito la ram occupata

Euclidean mAP
{'mAP': 0.40726871344372706, 'ie': 0.26161285473638807}
Cosine mAP
{'mAP': 0.4725439105296697, 'ie': 0.2592793254062691}

mAP using test as query on train
Euclidean mAP
{'mAP': 0.3482686371881832, 'ie': 0.25979275055117856}
Cosine mAP
{'mAP': 0.41990587567752924, 'ie': 0.2595528434992209}
bucket dispersion
{'perc': {'mean': 0.5177034708698071, 'deviation': 0.2733598467604444}, 'count': {'average': 20000.0, 'deviation': 8807.590306093944}, 'n_buckets': 6, 'n_items_counting_duplicates': 120000}
